<a href="https://colab.research.google.com/github/roberthsu2003/Transformer/blob/main/%E5%AF%A6%E6%88%B0%E9%81%8B%E7%94%A8/NER/ner1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate seqeval

1. 載入相關套件

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import evaluate
from datasets import load_dataset

2使用數據集

In [3]:
ner_datasets = load_dataset("peoples_daily_ner",cache_dir='./data',trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

peoples_daily_ner.py:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20865 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2319 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4637 [00:00<?, ? examples/s]

In [4]:
#查看第一筆資料
ner_datasets['train'][0]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

In [5]:
#查看ner_tags,5,6,所代表的意思
ner_datasets['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}

In [6]:
#取得label_list
label_list = ner_datasets['train'].features['ner_tags'].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

3數據預處理

In [7]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [8]:
#不正確,這是以字為單位,而不是以句子為單位
tokenizer(ner_datasets['train'][0]["tokens"])

{'input_ids': [[101, 3862, 102], [101, 7157, 102], [101, 3683, 102], [101, 6612, 102], [101, 1765, 102], [101, 4157, 102], [101, 1762, 102], [101, 1336, 102], [101, 7305, 102], [101, 680, 102], [101, 7032, 102], [101, 7305, 102], [101, 722, 102], [101, 7313, 102], [101, 4638, 102], [101, 3862, 102], [101, 1818, 102], [101, 511, 102]], 'token_type_ids': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], 'attention_mask': [[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]]}

In [9]:
tokenizer(ner_datasets['train'][0]['tokens'], is_split_into_words=True) #必需使用is_split_into_words

{'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
#英文的拆字,注意數量(10673, 12865, 12921, 8181, 8681)
res = tokenizer('interesting word')
res

{'input_ids': [101, 10673, 12865, 12921, 8181, 8681, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [11]:
# 0,0,0,0代表第1個字
# 1代表第2個字
res.word_ids()

[None, 0, 0, 0, 0, 1, None]

In [12]:
from pprint import pprint
def process_batch(examples):
    tokenized_examples = tokenizer(examples['tokens'], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_examples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_examples['labels'] = labels

    return tokenized_examples


In [13]:
tokenized_datasets = ner_datasets.map(process_batch,batched=True)
tokenized_datasets['train'][0]

Map:   0%|          | 0/20865 [00:00<?, ? examples/s]

Map:   0%|          | 0/2319 [00:00<?, ? examples/s]

Map:   0%|          | 0/4637 [00:00<?, ? examples/s]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0],
 'input_ids': [101,
  3862,
  7157,
  3683,
  6612,
  1765,
  4157,
  1762,
  1336,
  7305,
  680,
  7032,
  7305,
  722,
  7313,
  4638,
  3862,
  1818,
  511,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]}

In [14]:
from pprint import pprint
pprint(tokenized_datasets['train'][0])

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'id': '0',
 'input_ids': [101,
               3862,
               7157,
               3683,
               6612,
               1765,
               4157,
               1762,
               1336,
               7305,
               680,
               7032,
               7305,
               722,
               7313,
               4638,
               3862,
               1818,
               511,
               102],
 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'tokens': ['海',
            '钓',
            '比',
            '赛',
            '地',
            '点',
            '在',
            '厦',
            '门',
            '与',
            '金',
            '门',
            '之',
            '间',
            '的',
       

4建立模型

In [15]:
model = AutoModelForTokenClassification.from_pretrained("google-bert/bert-base-chinese",num_labels=len(label_list)) #預設是2,現在是7個

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model.num_labels

7

In [17]:
seqeval = evaluate.load('seqeval')
seqeval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [ ]:
import numpy as np

def eval_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)

    true_predictions =[
        [label_list[p] for p,l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
       for prediction, label in zip(predictions, labels)
    ]

    result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")
    return {
        "f1": result["overall_f1"]
    }

配置訓練參數

In [20]:
args = TrainingArguments(
    output_dir = "models_for_ner",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps = 50,
    num_train_epochs=3,
    report_to='none'
)

建立訓練器

In [21]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=eval_metric,
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
)

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss


ValueError: Found input variables with inconsistent numbers of samples:
[69, 35, 54, 56, 70, 18, 68, 70, 43, 23, 63, 63, 43, 28, 44, 63, 53, 51, 75, 28, 19, 18, 65, 45, 33, 42, 63, 57, 60, 71, 36, 126, 27, 26, 32, 103, 52, 56, 57, 39, 79, 55, 47, 30, 40, 28, 52, 21, 33, 41, 39, 50, 29, 15, 41, 58, 16, 93, 48, 36, 50, 68, 72, 55, 30, 16, 46, 25, 67, 65, 26, 38, 24, 37, 85, 56, 69, 26, 24, 45, 61, 34, 51, 57, 54, 49, 20, 37, 126, 26, 41, 44, 28, 28, 49, 49, 102, 30, 63, 55, 43, 15, 57, 35, 45, 25, 40, 49, 31, 43, 35, 35, 35, 23, 95, 89, 18, 70, 43, 55, 38, 52, 25, 38, 84, 33, 106, 28, 98, 47, 26, 16, 78, 15, 50, 50, 40, 43, 50, 38, 32, 43, 87, 51, 57, 59, 54, 126, 63, 62, 23, 16, 47, 32, 26, 107, 98, 26, 31, 115, 51, 72, 58, 35, 29, 20, 37, 18, 61, 24, 42, 21, 48, 59, 66, 32, 77, 39, 113, 16, 53, 28, 43, 28, 60, 70, 26, 34, 29, 28, 46, 25, 18, 29, 16, 73, 47, 21, 72, 58, 51, 42, 19, 62, 48, 13, 47, 69, 25, 42, 30, 28, 41, 57, 48, 46, 89, 49, 20, 91, 31, 102, 28, 38, 23, 36, 66, 75, 32, 5

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets['test'])

9模型預測

In [ ]:
from transformers import pipeline

ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)
ner_pipe("小明在台北上班")

In [ ]:
model.config

In [ ]:
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.config

In [ ]:
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)
ner_pipe("徐國堂在台北上班")

In [ ]:
tokenizer("徐國堂在台北上班")

In [ ]:
tokenizer.decode([2528, 1751, 1828])

In [ ]:
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)
res = ner_pipe("徐國堂在台北上班")
res

In [ ]:
ner_result = {}
x = '徐國堂在台北上班'
for r in res:
  if r["entity_group"] not in ner_datasets:
    ner_result[r['entity_group']] = []
  ner_result[r['entity_group']].append(x[r['start']:r['end']])
ner_result